# Intall Libraries, Run utlity functions, and import the queries (raw topics)

In [1]:
!pip install -U spacy
!pip install https://huggingface.co/kormilitzin/en_core_med7_trf/resolve/main/en_core_med7_trf-any-py3-none-any.whl
!pip install --upgrade ir_datasets
import spacy
import numpy as np
import pandas as pd
import ir_datasets

print('version',spacy.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 GB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.8 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
  Attempting uninstall: pydantic
    Found existing installation: 

## Example

In [2]:
nlp = spacy.load("en_core_med7_trf")
text = 'A patient was prescribed Magnesium hydroxide 400mg/5ml suspension PO of total 30ml bid for the next 5 days.'
doc = nlp(text)

for ent in doc.ents:
  print(ent.text, ent.label_)

Magnesium hydroxide DRUG
suspension FORM
PO ROUTE
30ml DOSAGE
bid FREQUENCY
for the next 5 days DURATION


## Load Topics

In [ ]:
# Queries
dataset = ir_datasets.load("clinicaltrials/2021/trec-ct-2022")
queries = pd.DataFrame(dataset.queries_iter())
del dataset
display(queries)

# Med7 implementation

In [9]:
def query_to_ne(query,ent_type):
  """
  extracts the necessary entities from the query
  """
  l = []
  proc = nlp(query)
  for ent in proc.ents:
    if ent.label_ == ent_type:
      l.append(ent.text)
  return l


top1 = pd.DataFrame(columns=["TOPIC","DRUG","DOSAGE","DURATION","FORM","FREQUENCY","ROUTE","STRENGTH"])
#print(topics.head())
list_topics = queries['text'].to_list()
#print(list_topics)
count = 0
d= {"TOPIC":0,"DRUG":[],"DOSAGE":[],"DURATION":[],"FORM":[],"FREQUENCY":[],"ROUTE":[],"STRENGTH":[]}
for topic in list_topics:
  count +=1
  d['TOPIC']= count
  proc = nlp(topic)
  for ent in proc.ents:
    if ent.label_ in d:
      d[ent.label_].append(ent.text)
  top1 = top1.append(d, ignore_index=True)
  d= {"TOPIC":0,"DRUG":[],"DOSAGE":[],"DURATION":[],"FORM":[],"FREQUENCY":[],"ROUTE":[],"STRENGTH":[]}


<ipython-input-9-a1fdbaf6106c>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top1 = top1.append(d, ignore_index=True)
<ipython-input-9-a1fdbaf6106c>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top1 = top1.append(d, ignore_index=True)
<ipython-input-9-a1fdbaf6106c>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top1 = top1.append(d, ignore_index=True)
<ipython-input-9-a1fdbaf6106c>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top1 = top1.append(d, ignore_index=True)
<ipython-input-9-a1fdbaf6106c>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

In [10]:
top1['DOSAGE'] = top1['DOSAGE'].apply(lambda y: np.nan if len(y)==0 else y)
top1['DURATION'] = top1['DURATION'].apply(lambda y: np.nan if len(y)==0 else y)
top1['FORM'] = top1['FORM'].apply(lambda y: np.nan if len(y)==0 else y)
top1['FREQUENCY'] = top1['FREQUENCY'].apply(lambda y: np.nan if len(y)==0 else y)
top1['ROUTE'] = top1['ROUTE'].apply(lambda y: np.nan if len(y)==0 else y)
top1['STRENGTH'] = top1['STRENGTH'].apply(lambda y: np.nan if len(y)==0 else y)
display(top1.head(200))

,TOPIC,DRUG,DOSAGE,DURATION,FORM,FREQUENCY,ROUTE,STRENGTH
0,1,[],NaN,NaN,NaN,NaN,NaN,NaN
1,2,[],NaN,NaN,NaN,NaN,NaN,NaN
2,3,[],NaN,NaN,NaN,NaN,NaN,NaN
3,4,[Tylenol],NaN,NaN,NaN,NaN,NaN,NaN
4,5,[],NaN,NaN,NaN,NaN,NaN,NaN
5,6,[],NaN,NaN,NaN,NaN,NaN,NaN
6,7,[],NaN,NaN,NaN,NaN,NaN,NaN
7,8,[],NaN,NaN,NaN,NaN,NaN,NaN
8,9,[],NaN,NaN,NaN,NaN,NaN,NaN
9,10,[],NaN,NaN,NaN,NaN,NaN,NaN
